In [ ]:
import sys
sys.path.append('/home/ashesh.ashesh/code/Disentangle')

In [ ]:
from IPython.display import display, HTML
import os

display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.data_loader.patch_index_manager import TilingMode

In [ ]:
ckpt_dir = "/group/jug/ashesh/training/disentangle/2406/D25-M3-S0-L8/17"
# ckpt_dir = '/group/jug/ashesh/training/disentangle/2406/D25-M3-S0-L8/4'
outputdir = '/group/jug/ashesh/EnsDeLyon/coverage_dicts/'

image_size_for_grid_centers = 32
mmse_count = 10
elem_size = 5
custom_image_size = None
data_t_list = [0] #[0,1,2,3]
skip_percentile = 50
tiling_mode = TilingMode.ShiftBoundary

save_comparative_plots =False
enable_calibration = False
batch_size = 16
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'
assert os.path.exists(ckpt_dir)

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
%run ./nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
if config.model.model_type == 25 and tokens[idx+1] == '2312':
    config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if 'depth3D' in config.data and config.data.depth3D > 1:
        config.data.mode_3D = True
        config.model.mode_3D = True
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.OnlyIba1P30
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

In [ ]:
dtype = config.data.data_type

if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
%run ./nb_core/disentangle_setup.ipynb

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = len(tar_tmp)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(min(ncols,len(inp_tmp))):
    ax[0,i].imshow(inp_tmp[i])

for channel_id in range(ncols):
    ax[1,channel_id].imshow(tar_tmp[channel_id])

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)

In [ ]:
# def get_patches(model, dset,num_workers=4, batch_size = 32):
#     dloader = DataLoader(dset, pin_memory=False, num_workers=num_workers, shuffle=False, batch_size=batch_size)
#     tar_patches = []
#     for batch in tqdm(dloader):
#         inp, tar = batch[:2]
#         inp = inp.cuda()
#         tar = tar.cuda()
#         tar_normalized = model.normalize_target(tar)
#         tar_patches.append(tar_normalized.cpu().numpy())
#     tar_patches = np.concatenate(tar_patches, axis=0)
#     return tar_patches


In [ ]:
from disentangle.analysis.background_threshold_estimation import get_background_thresholds
target_normalizer_func = lambda x: model.normalize_target(x.cuda())
background_thresholds = get_background_thresholds(val_dset,target_normalizer_func, num_workers=4, batch_size=32, skip_percentile=skip_percentile)
print(background_thresholds)

# import numpy as np
# tar_patches = get_patches(model, val_dset, num_workers=num_workers, batch_size=batch_size)
# background_thresholds = [np.percentile(tar_patches[:,i], skip_percentile) for i in range(tar_patches.shape[1])]

def background_patch_detection_func(data, ch_idx):
    threshold = background_thresholds[ch_idx]
    return data.reshape(data.shape[0],-1).mean(axis=-1) < threshold


In [ ]:
from disentangle.analysis.calibration_coverage import  get_calibration_coverage_data
import numpy as np
# mmse_count = 10
scalar_values = [1]
coverage_dict = {}
num_workers = 4
for scalar in scalar_values:
    var, err =get_calibration_coverage_data(model, val_dset, num_workers=num_workers,
                                       calib_stats_dict={0:{'scalar':scalar,'offset':0.0}, 
                                                                      1:{'scalar':scalar,'offset':0.0}, 
                                                                      2:{'scalar':scalar,'offset':0.0}, 
                                                                      },
                                        mmse_count=mmse_count,
                                        elem_size=elem_size,
                                        background_patch_detection_func=background_patch_detection_func)
    nan_mask = np.isnan(var).any(axis=-1).any(axis=-1)
    var = var[~nan_mask]
    err = err[~nan_mask]
    coverage_dict[scalar] = (var, err)

In [ ]:
nan_mask.shape

In [ ]:
var.shape, err.shape

### Grid search for optimal scaling parameter.
binary search inspired.

In [ ]:
from disentangle.analysis.calibration_coverage_grid_search import grid_search
from disentangle.analysis.calibration_coverage_grid_search import plot_coverage_plot
from disentangle.analysis.calibration_coverage_grid_search import get_percentage_occurance

factors = []
offsets = []
achieved_percentiles = []
for ch_idx in range(var.shape[1]):
    print('Starting the grid search')
    factor_ch, offset_ch, achieved_percentile_ch = grid_search(err[:,ch_idx], var[:,ch_idx], init_delta=10, init_factor=10,around_center=False)
    factors.append(factor_ch)
    offsets.append(offset_ch)
    achieved_percentiles.append(achieved_percentile_ch)
    
    print('')


In [ ]:
import seaborn as sns
_,ax = plt.subplots(figsize=(7,5))
log_scale = True
ch_idx = 2
sns.histplot(var[:,ch_idx,:4].reshape(-1,), bins=1000,log_scale=log_scale, ax=ax, label='Unscaled_var', stat='probability')
sns.histplot(factors[ch_idx]*var[:,ch_idx,:4].reshape(-1,) + offsets[ch_idx], bins=1000,log_scale=log_scale, ax=ax, label='Scaled_var', stat='probability')
sns.histplot(err[:,ch_idx], bins=1000,log_scale=log_scale, ax=ax, label='err', stat='probability')
plt.legend()

In [ ]:
data_left_oriented = plot_coverage_plot(var, err, factors, offsets, around_center=False)

In [ ]:
data_centered = plot_coverage_plot(var, err, factors, offsets, around_center=True)

In [ ]:
perfect = np.linspace(0,100,100)
for col_idx in range(var.shape[1]):
    act = data_centered[col_idx]['scaled'][1]
    mean_err = np.abs(perfect - act).mean() 
    max_err = np.abs(perfect - act).max()
    print(f'Centered: Channel {col_idx} MAE {mean_err:.2f}, MAX Err {max_err:.2f}')

In [ ]:
perfect = np.linspace(0,100,100)
for col_idx in range(var.shape[1]):
    act = data_left_oriented[col_idx]['scaled'][1]
    mean_err = np.abs(perfect - act).mean() 
    max_err = np.abs(perfect - act).max()
    print(f'LeftOriented: Channel {col_idx} MAE {mean_err:.2f}, MAX Err {max_err:.2f}')

In [ ]:
plt.plot(data_centered[2]['scaled'][2], label='Larger')
plt.plot(data_centered[2]['scaled'][0], label='Smaller')
plt.legend()